# MLP-Mixer Based GAN experiment  

 해당 노트북은 최근 연구가 많이 이뤄지는 MLP-Based Vision model을 사용해 Image-to-Image translation GAN을 만드는 실험 과정을 담았다.  

## MLP-Based models?  

 MLP-based model 이전에 비전 분야에서 가장 화두였던, 또한 현재도 굉장한 관심을 받는 모델인 **Transformer**부터 시작해야 MLP-Based model에 대해 이해 하기 쉽다. Transformer는 **Multi-Head-Attention**을 사용하는 모델을 의미한다. MHA같은 경우는 더 잘 설명된 [블로그](https://jalammar.github.io/illustrated-transformer/)가 많기에 여기선 설명하지 않고 넘어갈 것이다. Transformer는 본래 자연어 처리를 위해 고안된 모델이지만, CV 분야에서도 점차 연구(ViT, DeiT ...)된 모델이다. 이 중 CV 분야에서 Transformer를 처음으로 사용한 모델인 [ViT](https://arxiv.org/pdf/2010.11929.pdf)의 구조를 보고가자.  
 
![ViT](imgs/vit.png)  
<center>
    <em>Fig1. ViT Architecture</em>
</center></br>  

 ViT는 문장을 token으로 자르는 것 처럼, 이미지를 작은 조각인 patch로 자른 후 MHA/FFN을 반복하여 이미지에서 특징을 추출한다.

![ViT](imgs/vitbench.png)  
<center>
    <em>Fig2. ViT/BiT Benchmark</em>
</center></br>  

 ViT는 이전의 SOTA Model인 ResNet 기반 BiT와 비교해도 굉장히 좋은 성능을 보임을 *Fig.2*에서 확인 할 수 있다. 하지만 단순히 모든 상황에서 BiT를 능가하는 것이 아닌, pre-train에 사용되는 데이터가 방대해질수록 더 나은 성능을 보이는 것을 확인할 수 있다. 이는 Transformer가 CNN에 비해 더 낮은 **inductive bias**를 갖고 있기 때문이라고 주로 설명된다.  

 Inductive bias는 단순히 알고리즘의 특성에서 기인하는 모델이 답을 더 쉽게 찾기 위한 bias라고 생각하면 된다. 즉 inductive bias가 적다면 데이터셋의 Variance가 클 때 local minima로 빠질 가능성이 높지만 적당한 inductive bias가 있다면 global minima로 잘 수렴할 수 있을 것이다. 이렇게 설명하면 이해가 힘들 수도 있지만, 단순하게 이미지 처리에서 초창기의 MLP와 CNN을 비교해보면 이해하기가 쉽다. MLP는 'B H W C'의 이미지를 'B (H W C)'로 묶어서 사용했기에, 이미지 전체의 특성에서 특징을 찾아야 했지만 CNN은 'B H W C'의 데이터에서 특정 구간(**Kernel size = (h', w')**) 내의 패턴을 잡기만 하면 됐기에 더 좋은 성능을 보였다. 마찬가지로 MHA도 CNN과 달리 Local information에 대한 inductive bias가 모델 가장 앞의 Patch projection을 제외하면 전무하기에, 전체를 대표할 수 있을 정도로 엄청나게 많은 데이터를 필요로 했던 것이라 설명할 수 있다.  

![Mixer](imgs/mlpmixer.png)
<center>
    <em>Fig3. MLP-Mixer Architecture</em>
</center></br>  

 그럼 이번 실험의 주제인 MLP-Mixer로 돌아와 보자. MLP-Mixer의 기본 아이디어는 MLP에 patch projection으로 작은 inductive bias만 넣어주면 MHA처럼 높은 성능을 보일 수 있을까? 이다. 이를 위해 MLP-Mixer에서는 patch로 나눈 후 3D tensor로 변환된 'B (H' W') C' 이미지에서 '(H' W')'와 'C'차원을 transpose하며 MLP에 통과하는 것으로 MHA를 대체한다. 즉 단순히 '(H', W')'차원을 MLP에 통과시켜주는 것으로 MHA의 spatial간의 정보를 얻을 수 있다는 것이다. 

![Mixer](imgs/mlpmixerbench.png)
<center>
    <em>Fig4. MLP-Mixer/ViT Benchmark</em>
</center></br>  

 이러한 시도가 잘 통했는지 *Fig.4*를 보면 MLP-Mixer가 ViT에 비해 데이터셋이 커질수록 더 가파른 성능 증가를 보이기에 ViT에 비해 큰 포텐셜을 갖고 있다 평가했으며, 최근엔 [ResMLP](https://arxiv.org/abs/2105.03404), [CycleMLP](https://arxiv.org/abs/2107.10224), [gMLP](https://arxiv.org/abs/2105.08050) 등 다양한 MLP-Based model에 대한 연구가 이뤄지고 있다.  

## Why Image-to-Image Translation?  

![convMLP](imgs/mixerconvcompare.png)
<center>
    <em>Fig5. MLP-Mixer and ConvMLP Architecture Comparison</em>
</center></br>  

 MLP-Mixer는 2가지 구조적 한계점이 있는데, 첫 번째는 MHA처럼 Local information에 대한 inductive bias의 부족과, 두 번째는 *Fig.5*에서 보이는 것 처럼 Dimension이 일정하게 유지되기에 피라미드 구조가 요구되는 Object Detection / Segmentaion 분야에서 사용이 힘들다는 것이다. 하지만 Unpaired Image-to-Image Translation의 첫 주자인 CycleGAN에서 사용되는 ResNet기반의 아키텍쳐는 같은 dimension이 일정하게 유지되는 구조기에, Image-to-Image Trnaslation에 바로 MLP-Mixer를 사용할 수 있겠다는 생각이 들어서 이번 실험을 기획했다.   
 
 하지만 기획을 하면서 찾아보니 이미 해당 실험을 한 논문([MixerGAN](https://arxiv.org/pdf/2105.14110))이 있기에, 우선 실험을 진행한 후 나중에 해당 논문을 읽어보고 비교해 볼 것이다.

# CycleGAN  

우선 CycleGAN을 구현한 뒤 학습해보자. 앞서 언급한 ResNet 구조의 Generator를 사용할 것이며, 논문에서 소개하는 구조와 Loss를 그대로 사용할 것이다. 

In [1]:
import tensorflow as tf